# Exploratory Workflow

If you are looking at network data for the first time, use this workflow.

If you are updating a network diagram based on changes made from an earlier session, use the [Iteration Workflow](./iteration_workflow.ipynb).

In [126]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.style import StyleUtil

import py2cytoscape.util.cytoscapejs as cyjs
import py2cytoscape.cytoscapejs as renderer

import igraph as igraph
import pandas as pd
import json
import numpy as np

import sand.io as io
import sand.graph as sg
import sand.cytoscape.positions as scp

In [127]:
commit = "28cf190"
network_collection_name = "lein-topology"

In [128]:
# You might also want to use %cd magic to change to a different working directory.
data_path = "./data/" + network_collection_name + "-" + commit
edge_file = data_path + ".csv"
edgelist = io.csv_to_edgelist(edge_file)
g = sg.edgelist_to_igraph(edgelist)

In [129]:
g.summary()

'IGRAPH D-W- 107 206 -- \n+ attr: label (v), weight (e)'

In [130]:
# Access vertex attributes
g.vs[0]['label']

'topology.dependencies/dependencies'

In [131]:
g.is_weighted()

True

A loop is an edge for which both ends connect to a single vertex.
A pair of vertices with more than one edge between them is a multi-edge.

A multi-graph is a graph with loops or multi-edges.

A graph that is not a multi-graph is called a simple graph, and its edges are referred to as proper edges.

Checking whether or not a network is simple is an important preliminary step in doing a typical network analysis, as many models and methods assume the input graph to be simple or behave differently if it is not.

In [132]:
g.is_simple()

True

In [133]:
g.is_directed()

True

A DAG is a directed graph with no directed cycles.

In [134]:
g.is_dag()

True

A vertex $v$ in a graph $G$ is said to be _reachable_ from another vertex $u$ if there exists a walk from $u$ to $v$. A graph is said to be _connected_ if every vertex is reachable from every other. [SANDR - p 23] A connected graph with no cycles is called a _tree_.

A digraph $G$ is _weakly connected_ if its underlying graph (i.e., the result of stripping away the labels 'source' and 'target' from $G$) is connected.
It is called _strongly connected_ if every vertex $v$ is reachable from every $u$ by a directed walk. 

In [135]:
g.is_connected(mode="weak")

True

In [136]:
g.is_connected(mode="strong")

False

A common notion of distance between vertices on a graph is defined as the length of the shortest path(s) between the vertices. The value of the longest distance in a graph is called the _diameter_ of the graph.

In [137]:
g.diameter()

9

In [138]:
g.average_path_length()

3.6681974741676235

The following are vertex attributes computed from the network structure that we want to save and use in the visualization and analysis:

In [139]:
indegree  = g.degree(mode="in")
g.vs['indegree'] = indegree
outdegree = g.degree(mode="out")
g.vs['outdegree'] = outdegree

In [140]:
degrees = {v['label']: {'indegree': v['indegree'], 'outdegree': v['outdegree']} for v in g.vs}

## Properties of namespaces

### What is the degree distribution among vertex namespaces?

In [141]:
from itertools import groupby

sorted_degrees = sorted(degrees.items(), key=lambda tup: tup[0])

ns_degrees = {}
for key, group in groupby(sorted_degrees, lambda x: x[0].split('/')[0]):
    outdegrees = []
    indegrees = []
    for entry in group:
        outdegrees.append(entry[1]['outdegree'])
        indegrees.append(entry[1]['indegree'])
    ns_degrees[key] = {'indegree': sum(indegrees), 'outdegree': sum(outdegrees)}

In [142]:
ns_degrees

{'clojure.core': {'indegree': 152, 'outdegree': 0},
 'clojure.java.io': {'indegree': 1, 'outdegree': 0},
 'clojure.repl': {'indegree': 1, 'outdegree': 0},
 'clojure.string': {'indegree': 3, 'outdegree': 0},
 'clojure.test': {'indegree': 8, 'outdegree': 0},
 'clojure.tools.namespace.file': {'indegree': 2, 'outdegree': 0},
 'clojure.zip': {'indegree': 4, 'outdegree': 0},
 'example': {'indegree': 8, 'outdegree': 21},
 'java.util.Collections': {'indegree': 2, 'outdegree': 0},
 'leiningen.core.eval': {'indegree': 1, 'outdegree': 0},
 'leiningen.topology': {'indegree': 0, 'outdegree': 12},
 'org.clojure': {'indegree': 1, 'outdegree': 0},
 'topology.dependencies': {'indegree': 5, 'outdegree': 36},
 'topology.dependencies-test': {'indegree': 0, 'outdegree': 13},
 'topology.edgelist': {'indegree': 4, 'outdegree': 13},
 'topology.edgelist-test': {'indegree': 3, 'outdegree': 30},
 'topology.finder': {'indegree': 4, 'outdegree': 26},
 'topology.printer': {'indegree': 1, 'outdegree': 4},
 'topology

## Explore the network of namespaces

In [143]:
ns_edges = {}

from collections import namedtuple
Edge = namedtuple('Edge', ['source', 'target'])

def fqn_to_ns(fqn):
    return fqn.split('/')[0]

In [144]:
for e in edgelist:
    k = Edge(source=fqn_to_ns(e['source']), target=fqn_to_ns(e['target']))
    v = 1 if not ns_edges.has_key(k) else (ns_edges[k] + int(e['weight']))
    ns_edges[k] = v

In [145]:
ns_edges.items()[0:5]

[(Edge(source='leiningen.topology', target='leiningen.core.eval'), 1),
 (Edge(source='topology.dependencies', target='topology.qualifier'), 1),
 (Edge(source='topology.edgelist-test', target='java.util.Collections'), 1),
 (Edge(source='topology.edgelist-test', target='topology.edgelist'), 1),
 (Edge(source='topology.dependencies-test', target='clojure.string'), 1)]

In [146]:
ns_vertices = set()
for e in ns_edges.keys():
    ns_vertices.add(e.source)
    ns_vertices.add(e.target)

In [147]:
ns_vertices

{'clojure.core',
 'clojure.java.io',
 'clojure.repl',
 'clojure.string',
 'clojure.test',
 'clojure.tools.namespace.file',
 'clojure.zip',
 'example',
 'java.util.Collections',
 'leiningen.core.eval',
 'leiningen.topology',
 'org.clojure',
 'topology.dependencies',
 'topology.dependencies-test',
 'topology.edgelist',
 'topology.edgelist-test',
 'topology.finder',
 'topology.printer',
 'topology.qualifier',
 'topology.symbols'}

In [148]:
ns_graph = igraph.Graph(directed=True)
ns_graph.add_vertices(list(ns_vertices))

for e in ns_edges.items():
    ns_graph.add_edge(e[0].source, e[0].target, weight=e[1], directed=True)

In [149]:
ns_graph.summary()

'IGRAPH DNW- 20 40 -- \n+ attr: name (v), directed (e), weight (e)'

In [150]:
# Create a list of patterns for all namespaces that we want to keep:
ns_names_to_keep = ['topology', 'clojure.java.io', 'clojure.repl', 'clojure.tools.namespace.file', 'clojure.zip']

ns_interest = ns_graph.vs(lambda v: any(match in v['name'] for match in ns_names_to_keep))

ns_subgraph = ns_graph.subgraph(ns_interest)
ns_subgraph.simplify() # Remove loops
ns_subgraph.summary()

'IGRAPH DN-- 13 12 -- \n+ attr: name (v)'

## Load into Cytoscape with a default layout

In [151]:
# Create py2cytoscape client
cy = CyRestClient()

In [152]:
cy.session.delete()

In [153]:
ns_network = cy.network.create_from_igraph(ns_subgraph, name="namespaces", collection=network_collection_name)

In [154]:
ns_network_id = ns_network.get_id()
ns_network_id

52

In [155]:
# Apply layout
cy.layout.apply(name='force-directed', network=ns_network)

## Apply Style

In [156]:
# Get a reference to the existing style
curved = cy.style.create('Curved')

In [157]:
cy.style.apply(curved, ns_network)

## Explore in Cytoscape

At this point, we have a list of interesting namespaces laid out in Cytoscape. We can now start visually exploring to look for interesting features to dig into more deeply.

When this step is done, we can move on to looking at the function network beyond just the namespaces.

## Extract the subgraph of local namespaces from the full graph

There are some analyses where it will be useful to see all the vertices. For the high-level architecture diagram, we can focus on the library's namespaces.

In [158]:
# List all patterns of vertex names that we want to keep:
names_to_keep = ['topology', 'clojure.core/*err*', 'clojure.core/println']

In [159]:
lv = g.vs(lambda v: any(match in v['label'] for match in names_to_keep))

# lg...the local graph
lg = g.subgraph(lv, implementation='copy_and_delete')

In [160]:
# Copy the label attribute to name so that cytoscape will pick it up without extra mapping
lg.vs['name'] = lg.vs['label']
lg.summary()

'IGRAPH DNW- 26 27 -- \n+ attr: indegree (v), label (v), name (v), outdegree (v), weight (e)'

In [161]:
# Visualize in Cytoscape
fn_network = cy.network.create_from_igraph(lg, name=commit, collection=network_collection_name)
cy.layout.apply(name='force-directed', network=fn_network)
cy.style.apply(curved, fn_network)

### Map attributes to visual properties

In [162]:
degrees = fn_network.get_node_column('outdegree')

# Scale color of nodes
color_gradient = StyleUtil.create_2_color_gradient(min=1, max=degrees.max(), colors=('white', '#FFCC00'))
curved.create_continuous_mapping(column='outdegree', vp='NODE_FILL_COLOR', col_type='Double', points=color_gradient)

# Scale size of nodes
degree_to_size = StyleUtil.create_slope(min=0, max=degrees.max(), values=(30, 80))
curved.create_continuous_mapping(column='outdegree', vp='NODE_HEIGHT', col_type='Double', points=degree_to_size)
curved.create_continuous_mapping(column='outdegree', vp='NODE_WIDTH', col_type='Double', points=degree_to_size)
# curved.create_continuous_mapping(column='outdegree', vp='NODE_LABEL_FONT_SIZE', col_type='Double', points=degree_to_size)

In [163]:
# BUG: All weights are 1 because igraph's subgraph method loses the weight attribute.
weights = fn_network.get_edge_column('weight')

weight_to_size = StyleUtil.create_slope(min=weights.min(), max=weights.max(), values=(2,10))
curved.create_continuous_mapping(column='weight', vp="EDGE_WIDTH", col_type='Double', points=weight_to_size)

In [165]:
cy.style.apply(curved, fn_network)

# Note that there are still a couple of manual steps using the default Curved.
# Uncheck 'Lock node width and height'
# Remove the default size mapping

## Save the updated layout coordinates after making changes

One benefit of this workflow over solutions that just render static diagrams is the ability to make changes manually to the network layout in Cytoscape.

After making changes, save the coordinates for a later session using the [Iteration Workflow](./iteration_workflow.ipynb).

In [166]:
positions_file = data_path + "-positions.csv"
scp.positions_to_csv(network=fn_network, path=positions_file)

You can now safely close Cytoscape.

## Future Work

### Interpretation of articulation points

A vertex is an articulation point if its removal increases the number of connected components in the graph:

In [167]:
lg.vs(lg.articulation_points())['label']

['topology.dependencies/dependencies',
 'topology.finder/find-sources-in-dir',
 'topology.dependencies/ns->fn-dep-map',
 'topology.finder/source-paths->namespaces',
 'topology.finder/dirs->sources',
 'topology.symbols/symbols',
 'topology.edgelist-test/edges',
 'topology.edgelist/dirs->fn-edges',
 'topology.symbols/zip-nodes',
 'topology.edgelist/ns->edgelist']

What are the implications of these articulation points in the domain of function dependency graphs? We'd need to look at several examples across multiple applications. In the context of a larger system architecture, these might indicate single points of failure that would partition the system.